### 문제 6-1 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용하기)

In [1]:
# %pip install langchain-community langchain-openai pydantic faiss-cpu langgraph python-dotenv

In [2]:
import os
import re
from enum import Enum
from typing import List, TypedDict, Literal

from dotenv import load_dotenv

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langgraph.graph import StateGraph, END

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

print("--- 카페 메뉴 추천 시스템 구축 시작 ---")

# --- 0. 초기 설정: LLM 및 임베딩 모델 ---
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7
)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536
)
print(f"OpenAI LLM 모델 '{llm.model_name}' 및 임베딩 모델 '{embeddings.model}' 설정 완료.")


--- 카페 메뉴 추천 시스템 구축 시작 ---
OpenAI LLM 모델 'gpt-3.5-turbo' 및 임베딩 모델 'text-embedding-3-small' 설정 완료.


In [3]:

# --- 1. 카페 메뉴 데이터셋 및 Vector DB 구축 ---
cafe_menu_data = [
    {"name": "아메리카노", "price": "₩4,500", "description": "진한 에스프레소에 물을 더한 클래식 커피. 산미와 바디감의 균형이 좋습니다."},
    {"name": "카페 라떼", "price": "₩5,000", "description": "부드러운 우유와 에스프레소가 조화로운 커피. 고소하고 부드러운 맛을 선호하는 분께 추천합니다."},
    {"name": "바닐라 라떼", "price": "₩5,500", "description": "달콤한 바닐라 시럽이 들어간 라떼. 달콤하면서도 커피의 풍미를 잃지 않습니다."},
    {"name": "카라멜 마키아또", "price": "₩6,000", "description": "바닐라 시럽, 스팀 우유, 에스프레소 위에 카라멜 드리즐이 올라간 달콤한 커피."},
    {"name": "콜드브루", "price": "₩5,500", "description": "저온에서 장시간 추출하여 부드럽고 깔끔한 맛이 특징인 커피. 산미가 적고 초콜릿 풍미가 느껴집니다."},
    {"name": "자몽 허니 블랙티", "price": "₩6,000", "description": "상큼한 자몽과 달콤한 꿀, 그리고 깊은 홍차의 조화가 이색적인 티 음료. 카페인 부담 없이 즐길 수 있습니다."},
    {"name": "제주 유기농 말차 라떼", "price": "₩6,500", "description": "제주산 유기농 말차를 사용하여 진하고 고소한 풍미가 일품인 라떼. 달콤쌉쌀한 맛이 매력적입니다."},
    {"name": "초콜릿 라떼", "price": "₩5,500", "description": "진한 다크 초콜릿과 우유가 만나 부드럽고 달콤한 맛을 선사합니다. 남녀노소 누구나 좋아하는 음료."},
    {"name": "딸기 요거트 스무디", "price": "₩6,500", "description": "새콤달콤한 딸기와 요거트가 어우러진 시원한 스무디. 과일의 상큼함을 그대로 느낄 수 있습니다."},
    {"name": "레몬 마들렌", "price": "₩3,000", "description": "상큼한 레몬 향이 가득한 부드러운 구움 과자. 커피와 함께 곁들이기 좋습니다."},
    {"name": "플레인 스콘", "price": "₩3,500", "description": "겉은 바삭하고 속은 촉촉한 기본 스콘. 잼이나 클로티드 크림과 잘 어울립니다."}
]

menu_documents = []
for item in cafe_menu_data:
    doc = Document(
        page_content=f"메뉴: {item['name']}\n가격: {item['price']}\n설명: {item['description']}",
        metadata={"menu_name": item['name'], "price": item['price'], "description": item['description']}
    )
    menu_documents.append(doc)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(menu_documents)

menu_db = FAISS.from_documents(documents=splits, embedding=embeddings)
print("카페 메뉴 데이터로 FAISS 벡터 DB 구축 완료.")


카페 메뉴 데이터로 FAISS 벡터 DB 구축 완료.


In [4]:

# --- 2. 상태 정의 (LangGraph State) ---
class QueryType(str, Enum): # QueryType을 AgentState 밖으로 이동하여 독립적으로 정의
    MENU = "메뉴문의"
    PRICE = "가격문의"
    RECOMMENDATION = "추천요청"
    OTHER = "기타"

class AgentState(TypedDict):
    messages: List[BaseMessage]
    # 수정: 분류된 질문 유형을 저장할 새 필드 추가
    query_type: QueryType # 분류된 질문 유형

# --- 3. 고급 정보 추출 함수 ---
def extract_menu_info(doc: Document) -> dict:
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    price = doc.metadata.get('price', '가격 정보 없음')
    description = doc.metadata.get('description', '설명 없음')

    if price == '가격 정보 없음':
        price_match = re.search(r'₩([\d,]+)', content)
        if price_match:
            price = price_match.group(0)

    if description == '설명 없음':
        description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
        if description_match:
            description = description_match.group(1).strip()
    
    return {
        "name": menu_name,
        "price": price,
        "description": description
    }


In [5]:

# --- 4. 문의 분류 로직 (LangGraph Node로 동작하도록 수정) ---
def classify_query(state: AgentState) -> AgentState:
    """사용자 메시지를 기반으로 질문 유형을 분류하고 상태를 업데이트합니다."""
    last_message_content = state["messages"][-1].content.lower()
    
    query_type: QueryType

    if "메뉴" in last_message_content or "뭐" in last_message_content or "종류" in last_message_content:
        query_type = QueryType.MENU
    elif "가격" in last_message_content or "얼마" in last_message_content or "비싸" in last_message_content:
        query_type = QueryType.PRICE
    elif "추천" in last_message_content or "뭐가 좋아" in last_message_content or "인기" in last_message_content:
        query_type = QueryType.RECOMMENDATION
    else:
        query_type = QueryType.OTHER
    
    print(f"[DEBUG] 질문 유형 분류됨: {query_type.value}")
    # 수정: 딕셔너리 형태로 상태를 반환하여 'query_type' 필드를 업데이트합니다.
    return {"query_type": query_type}


In [6]:

# --- 5. 응답 생성 함수들 (이전과 동일, 인자 타입만 AgentState로 통일) ---

def handle_menu_query(state: AgentState) -> AgentState:
    user_message = state["messages"][-1].content
    print(f"[DEBUG] 메뉴 문의 처리 중: '{user_message}'")

    docs = menu_db.similarity_search(user_message, k=4)
    
    if not docs:
        ai_message = AIMessage(content="죄송합니다. 요청하신 메뉴 정보를 찾을 수 없습니다. 다른 메뉴를 말씀해주시겠어요?")
    else:
        extracted_info_list = [extract_menu_info(doc) for doc in docs]
        
        context_str = "\n".join([
            f"메뉴: {info['name']}\n가격: {info['price']}\n설명: {info['description']}"
            for info in extracted_info_list
        ])
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """당신은 친절한 카페 직원입니다. 고객이 문의한 메뉴에 대해, 제공된 정보를 바탕으로 상세하고 친절하게 설명해주세요. 
             특히, 메뉴 이름, 가격, 그리고 설명을 포함하여 답변해주세요. 여러 메뉴가 검색되면 모두 설명해주세요."""),
            ("user", f"다음 메뉴 정보들을 바탕으로 '{user_message}'에 대해 답변해주세요:\n\n{context_str}")
        ])
        
        chain = prompt | llm
        response_content = chain.invoke({"context_str": context_str, "user_message": user_message}).content
        ai_message = AIMessage(content=response_content)

    return {"messages": state["messages"] + [ai_message]}

def handle_price_query(state: AgentState) -> AgentState:
    user_message = state["messages"][-1].content
    print(f"[DEBUG] 가격 문의 처리 중: '{user_message}'")

    docs = menu_db.similarity_search(user_message, k=5)

    if not docs:
        ai_message = AIMessage(content="죄송합니다. 가격 정보를 찾을 수 없습니다. 어떤 메뉴의 가격이 궁금하신가요?")
    else:
        extracted_info_list = [extract_menu_info(doc) for doc in docs]
        
        context_str = "\n".join([
            f"메뉴: {info['name']}, 가격: {info['price']}"
            for info in extracted_info_list
            if info['price'] != '가격 정보 없음'
        ])
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """당신은 친절한 카페 직원입니다. 고객이 문의한 메뉴의 가격 정보에 대해, 제공된 정보를 바탕으로 명확하게 답변해주세요. 여러 메뉴가 검색되면 모두 포함해서 답변해주세요."""),
            ("user", f"다음 메뉴 가격 정보들을 바탕으로 '{user_message}'에 대해 답변해주세요:\n\n{context_str}")
        ])
        
        chain = prompt | llm
        response_content = chain.invoke({"context_str": context_str, "user_message": user_message}).content
        ai_message = AIMessage(content=response_content)

    return {"messages": state["messages"] + [ai_message]}

def handle_recommendation_request(state: AgentState) -> AgentState:
    user_message = state["messages"][-1].content
    print(f"[DEBUG] 추천 요청 처리 중: '{user_message}'")

    docs = menu_db.similarity_search(user_message, k=3)
    if not docs:
        docs = menu_db.similarity_search("인기 메뉴", k=3)

    if not docs:
        ai_message = AIMessage(content="죄송합니다. 지금 추천해 드릴 만한 메뉴를 찾을 수 없습니다. 어떤 종류의 음료를 선호하시나요?")
    else:
        extracted_info_list = [extract_menu_info(doc) for doc in docs]
        
        context_str = "\n".join([
            f"메뉴: {info['name']}\n가격: {info['price']}\n설명: {info['description']}"
            for info in extracted_info_list
        ])
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """당신은 고객의 취향을 파악하여 메뉴를 추천하는 전문 바리스타입니다. 
             제공된 메뉴 정보를 바탕으로 고객의 요청에 맞춰 메뉴를 추천하고, 추천 이유를 친절하게 설명해주세요. 
             여러 메뉴를 추천할 수도 있습니다."""),
            ("user", f"다음 메뉴 정보들을 바탕으로 '{user_message}'에 대해 추천해주세요:\n\n{context_str}")
        ])
        
        chain = prompt | llm
        response_content = chain.invoke({"context_str": context_str, "user_message": user_message}).content
        ai_message = AIMessage(content=response_content)
        
    return {"messages": state["messages"] + [ai_message]}

def handle_other_queries(state: AgentState) -> AgentState:
    user_message = state["messages"][-1].content
    print(f"[DEBUG] 기타 문의 처리 중: '{user_message}'")

    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 친절한 카페 직원입니다. 고객의 일반적인 질문에 대해 친절하게 답변해주세요. 특정 메뉴나 가격 정보가 아닌 일반적인 질문입니다."),
        ("user", f"고객 질문: {user_message}")
    ])
    
    chain = prompt | llm
    response_content = chain.invoke({"user_message": user_message}).content
    ai_message = AIMessage(content=response_content)
    
    return {"messages": state["messages"] + [ai_message]}


In [7]:

# --- 6. LangGraph 그래프 정의 및 실행 ---
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("classify", classify_query) # classify_query가 이제 딕셔너리를 반환
workflow.add_node("handle_menu", handle_menu_query)
workflow.add_node("handle_price", handle_price_query)
workflow.add_node("handle_recommendation", handle_recommendation_request)
workflow.add_node("handle_other", handle_other_queries)

# 시작 지점 설정
workflow.set_entry_point("classify")

# 조건부 엣지 설정: 'classify' 노드가 반환하는 'query_type' 필드를 기준으로 분기
workflow.add_conditional_edges(
    "classify",
    lambda state: state["query_type"], # classify 노드가 업데이트한 'query_type' 필드 값을 기준으로 분기
    {
        QueryType.MENU: "handle_menu",
        QueryType.PRICE: "handle_price",
        QueryType.RECOMMENDATION: "handle_recommendation",
        QueryType.OTHER: "handle_other",
    }
)

# 각 응답 처리 노드에서 END로 연결하여 그래프 종료
workflow.add_edge("handle_menu", END)
workflow.add_edge("handle_price", END)
workflow.add_edge("handle_recommendation", END)
workflow.add_edge("handle_other", END)

# 그래프 컴파일
app = workflow.compile()
print("LangGraph 애플리케이션 컴파일 완료.")


LangGraph 애플리케이션 컴파일 완료.


In [ ]:

# --- 7. 대화 테스트 ---
print("\n--- 대화 시작 (종료하려면 'exit' 입력) ---")
messages = [] # 대화 이력은 매 턴마다 업데이트될 것이므로 빈 리스트로 시작

while True:
    user_input = input("고객님: ")
    if user_input.lower() == 'exit':
        print("대화를 종료합니다.")
        break
    
    # LangGraph에 전달할 초기 상태는 현재 대화 이력만 포함
    # classify_query 노드가 query_type을 추가할 것임
    current_state = {"messages": messages + [HumanMessage(content=user_input)]}

    # LangGraph 실행
    response = app.invoke(current_state)
    
    # 최종 상태의 메시지 리스트를 가져와서 다음 턴을 위해 업데이트
    messages = response["messages"]
    
    # 마지막 AI 메시지 추출 및 출력
    ai_response = messages[-1]
    print(f"카페 챗봇: {ai_response.content}")


--- 대화 시작 (종료하려면 'exit' 입력) ---
